import libraries

In [2]:
import pandas as pd
import requests
import config


In [3]:

url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/EHAM/2023-03-28T20:00/2023-03-29T08:00"

querystring = {"direction":"Arrival","withCancelled":"true","withCodeshared":"false","withLocation":"false"}

headers = {
	"X-RapidAPI-Key": config.flights_API_key,
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

# print(response.text) # uncomment to see the raw data

In [10]:
response_arrivals = response.json()



In [11]:
response_arrivals['arrivals'][0].keys()

dict_keys(['movement', 'number', 'status', 'codeshareStatus', 'isCargo', 'aircraft', 'airline'])

In [12]:
#accessing local time arrival
response_arrivals['arrivals'][0]['movement']['scheduledTimeLocal']

'2023-03-28 10:30+02:00'

In [53]:
response_arrivals['arrivals'][0]

{'movement': {'airport': {'icao': 'UBBB', 'iata': 'GYD', 'name': 'Baku'},
  'scheduledTimeLocal': '2023-03-28 10:30+02:00',
  'actualTimeLocal': '2023-03-28 20:02+02:00',
  'runwayTimeLocal': '2023-03-28 20:02+02:00',
  'scheduledTimeUtc': '2023-03-28 08:30Z',
  'actualTimeUtc': '2023-03-28 18:02Z',
  'runwayTimeUtc': '2023-03-28 18:02Z',
  'quality': ['Basic', 'Live']},
 'number': '7L 211',
 'status': 'Arrived',
 'codeshareStatus': 'IsOperator',
 'isCargo': True,
 'aircraft': {'model': 'Boeing 747-400 Freighter'},
 'airline': {'name': 'Silk Way West', 'iata': '7L', 'icao': 'AZG'}}

In [21]:
#accessing city name
response_arrivals['arrivals'][0]['movement']['airport']['name']

'Baku'

In [25]:
# accessing ICAO code
response_arrivals['arrivals'][0]['movement']['airport']['icao']

'UBBB'

In [82]:
response_arrivals['arrivals'][0]['movement']['actualTimeLocal']

'2023-03-28 20:02+02:00'

In [8]:
print(response.json)

<bound method Response.json of <Response [200]>>


In [13]:
from IPython.display import JSON

In [ ]:
display(JSON(response.json()))

dataframes for cities and airports

In [15]:
arrivals  = pd.json_normalize(response.json()['arrivals'])

In [150]:
arrivals[arrivals['movement.airport.icao'].isin(['EDDF', 'EDDS', 'EDDB', 'UCFM'])]

,number,status,codeshareStatus,isCargo,movement.airport.icao,movement.airport.iata,movement.airport.name,movement.scheduledTimeLocal,movement.actualTimeLocal,movement.runwayTimeLocal,...,airline.name,airline.iata,airline.icao,callSign,movement.runway,aircraft.reg,aircraft.modeS,movement.terminal,movement.baggageBelt,city_id
16,KL 1774,Arrived,IsOperator,False,EDDF,FRA,Frankfurt-am-Main,2023-03-28 20:25+02:00,2023-03-28 20:20+02:00,2023-03-28 20:20+02:00,...,KLM,KL,KLM,KLM1774,NaN,PH-EXS,485779,2,8,16
63,LH 1002,Arrived,IsOperator,False,EDDF,FRA,Frankfurt-am-Main,2023-03-28 22:15+02:00,2023-03-28 22:23+02:00,2023-03-28 22:23+02:00,...,Lufthansa,LH,DLH,DLH5AK,18R,D-AIZD,3C6744,1,2,63
84,KL 1834,Arrived,IsOperator,False,EDDB,BER,Berlin,2023-03-28 22:05+02:00,2023-03-28 21:50+02:00,2023-03-28 21:50+02:00,...,KLM,KL,KLM,KLM70D,18R,PH-EZZ,485065,2,12,84
134,KL 1818,Arrived,IsOperator,False,EDDB,BER,Berlin,2023-03-29 07:20+02:00,2023-03-29 07:11+02:00,2023-03-29 07:11+02:00,...,KLM,KL,KLM,KLM44R,18R,PH-BXU,484442,2,11,134
136,KL 1866,Arrived,IsOperator,False,EDDS,STR,Stuttgart,2023-03-29 07:25+02:00,2023-03-29 07:30+02:00,2023-03-29 07:30+02:00,...,KLM,KL,KLM,KLM68W,NaN,D-AZFA,3C1AEB,2,8,136
161,KL 1762,Arrived,IsOperator,False,EDDF,FRA,Frankfurt-am-Main,2023-03-29 08:10+02:00,2023-03-29 07:57+02:00,2023-03-29 07:57+02:00,...,KLM,KL,KLM,KLM1762,NaN,PH-EZW,484F7F,2,12,161


In [148]:
#BETTER WAY TO DO THIS
# creating airport name and ICAO code dataframe with chaining method
airports_df = arrivals[arrivals['movement.airport.icao'].isin(['EDDF', 'LTFM'])][['movement.airport.icao', 'movement.airport.name']]\
    .drop_duplicates().rename(columns={'movement.airport.icao': 'icao_airport', 'movement.airport.name': 'city_name'}).reset_index(drop=True)

In [155]:
airports_df

,icao_airport,city_name
0,EDDF,Frankfurt-am-Main
1,LTFM,Istanbul


In [157]:
airports_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   icao_airport  2 non-null      object
 1   city_name     2 non-null      object
dtypes: object(2)
memory usage: 160.0+ bytes


In [156]:
# create a new df cities with the cities based on arrivals df filtering movement.airport.icao and airport.name columns and renaming them into new columns icao and city

arrivals[arrivals['movement.airport.icao'].isin(['EDDF', 'LTFM'])][['movement.airport.icao', 'movement.airport.name']].drop_duplicates()

cities_df = arrivals[['movement.airport.icao', 'movement.airport.name']].drop_duplicates()



# rename columns as icao and city
cities_df = pd.DataFrame({'city': ['Frankfurt', 'Istanbul'], 'icao': ['EDDF', 'LTFM']})
cities_df

,city,icao
0,Frankfurt,EDDF
1,Istanbul,LTFM


In [13]:
arrivals.sample(5)

,number,callSign,status,codeshareStatus,isCargo,movement.airport.icao,movement.airport.iata,movement.airport.name,movement.scheduledTimeLocal,movement.actualTimeLocal,...,movement.runway,movement.quality,aircraft.reg,aircraft.modeS,aircraft.model,airline.name,airline.iata,airline.icao,movement.terminal,movement.baggageBelt
31,FR 2334,NaN,Arrived,IsOperator,False,LEMG,AGP,Málaga,2023-03-28 21:25+02:00,2023-03-28 21:34+02:00,...,NaN,"[Basic, Live]",NaN,NaN,Boeing 737-800 (winglets),Ryanair,FR,RYR,4,22
145,KL 1866,KLM68W,Arrived,IsOperator,False,EDDS,STR,Stuttgart,2023-03-29 07:25+02:00,2023-03-29 07:30+02:00,...,NaN,"[Basic, Live]",D-AZFA,3C1AEB,Embraer 190,KLM,KL,KLM,2,8
36,KL 1150,KLM54B,Arrived,IsOperator,False,ENGM,OSL,Oslo,2023-03-28 21:05+02:00,2023-03-28 21:03+02:00,...,NaN,"[Basic, Live]",PH-EZB,484ACC,Embraer 190,KLM,KL,KLM,2,12
44,KL 994,KLM30B,Arrived,IsOperator,False,EGLC,LCY,London,2023-03-28 21:20+02:00,2023-03-28 20:56+02:00,...,NaN,"[Basic, Live]",PH-EZN,484C52,Embraer 190,KLM,KL,KLM,2,13
111,KL 652,KLM292,Arrived,IsOperator,False,KIAD,IAD,District of Columbia,2023-03-29 06:45+02:00,2023-03-29 06:21+02:00,...,18R,"[Basic, Live]",PH-AKA,484F71,Airbus A330-300,KLM,KL,KLM,2,12


arrivals dataFrame with one city

In [74]:
airport = 'EDDS'
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport}/2023-03-28T20:00/2023-03-29T08:00"

#alternative way to create url, where you directly insert the airport code
# url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/EDDS/2023-03-28T20:00/2023-03-29T08:00"

querystring = {"direction":"Arrival","withCancelled":"true","withCodeshared":"false","withLocation":"false"}

headers = {
	"X-RapidAPI-Key": config.flights_API_key,
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

In [ ]:
city_response = response.json()
# city_response # uncomment to see the raw data

In [32]:
city_response['arrivals'][0]



{'movement': {'airport': {'icao': 'UBBB', 'iata': 'GYD', 'name': 'Baku'},
  'scheduledTimeLocal': '2023-03-28 10:30+02:00',
  'actualTimeLocal': '2023-03-28 20:02+02:00',
  'runwayTimeLocal': '2023-03-28 20:02+02:00',
  'scheduledTimeUtc': '2023-03-28 08:30Z',
  'actualTimeUtc': '2023-03-28 18:02Z',
  'runwayTimeUtc': '2023-03-28 18:02Z',
  'quality': ['Basic', 'Live']},
 'number': '7L 211',
 'status': 'Arrived',
 'codeshareStatus': 'IsOperator',
 'isCargo': True,
 'aircraft': {'model': 'Boeing 747-400 Freighter'},
 'airline': {'name': 'Silk Way West', 'iata': '7L', 'icao': 'AZG'}}

arrivals dataFrame with multiple cities

In [ ]:
#an example how to filter data with if statement
# arrivals = []
# for flight in json_data[0]:
#     if flight['movement']['airport']['iata'] == 'STR': #and flight['status'] == 'Arrived':
#         arrivals.append({
#             'Departure Airport': flight['movement']['airport']['name'],
#             'Actual Arrival': flight['movement']['actualTimeLocal'],
#             'Scheduled Arrival': flight['movement']['scheduledTimeLocal']
#         })

# # Create DataFrame
# arrivals_df_two = pd.DataFrame(arrivals)

# Print DataFrame


In [150]:
# icao codes for airports frankfurt, istanbul
airports = ['EDDF', 'LTFM']

# create a list of dictionaries
arrivals_dict = {
    'icao': [],
    'city': [],
    'actual_arrival': [],
    'scheduled_arrival': [],
    'terminal': [],
    'airline': []
}

# loop through the airports list
for airport in airports:
    # the url, querystring and headers were created in rapidapi website and copied here
    # url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport}/2023-03-28T20:00/2023-03-29T08:00"
    # querystring = {"direction":"Arrival","withCancelled":"true","withCodeshared":"false","withLocation":"false"}
    
    # headers = {
    #     "X-RapidAPI-Key": config.flights_API_key,
    #     "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    # }
    # make the request
    response = requests.request("GET", url, headers=headers, params=querystring)
    # get the json data
    
    json_data = response.json()['arrivals']
    
    for flight in json_data:
        arrivals_dict['icao'].append(flight['movement']['airport'].get("icao", None))
        arrivals_dict['city'].append(flight['movement']['airport'].get('name', None))
        arrivals_dict['actual_arrival'].append(flight['movement'].get('name', None))
        arrivals_dict['scheduled_arrival'].append(flight['movement'].get('scheduledTimeLocal', None))
        arrivals_dict['terminal'].append(flight['movement'].get('terminal', None))
        arrivals_dict['airline'].append(flight['airline'].get('name', None))    
    

    
    # json_data = response.json()

    # loop through the json data
    # for flight in json_data['arrivals']:
        # arrivals_dict['icao'].append(flight['movement']['airport']['icao'])
        # arrivals_dict['city'].append(flight['movement']['airport']['name'])
        # arrivals_dict['actual_arrival'].append(flight['movement']['actualTimeLocal'])
        # arrivals_dict['scheduled_arrival'].append(flight['movement']['scheduledTimeLocal'])
        # arrivals_dict['terminal'].append(flight['movement']['terminal'])
        # arrivals_dict['airline'].append(flight['airline']['name'])

        # arrivals_flights_dict = {}
        # arrivals_flights_dict['arrival_icao'] = airport
        # arrivals_flights_dict['scheduled_flight'] = flight[0]['movement']['scheduledTimeLocal']

    # # alternatively, you can use (i in range(len(json_data['arrivals']))):
    # for i in range(len(json_data['arrivals'])):
    #     arrivals_dict['icao'] = i['movement']['airport'].get('icao', None)
        # arrivals_dict['city'].append(json_data['arrivals'][i]['movement']['airport']['name'])
        # arrivals_dict['actual_arrival'].append(json_data['arrivals'][i]['movement']['actualTimeLocal'])
        # arrivals_dict['scheduled_arrival'].append(json_data['arrivals'][i]['movement']['scheduledTimeLocal'])
        # arrivals_dict['terminal'].append(json_data['arrivals'][i]['movement']['terminal'])
        # arrivals_dict['airline'].append(json_data['arrivals'][i]['airline']['name'])

# create the DataFrame
arrivals_df = pd.DataFrame(arrivals_dict)

In [36]:
#THIS VERSION HAS ARRIVAL AIRPORTS
#indentation is very important, otherwiser some elements not included in the dictionary and ger overwritten by the next iteration

# icao codes for airports frankfurt, istanbul, stuttgart
airports = ['EDDF', 'LTFM']

arrivals_dict = {
    'icao': [],
    'city': [],
    'actual_arrival': [],
    'scheduled_arrival': [],
    'terminal': [],
    'airline': [],
    'airport': []
    }

# loop through the airports list
for airport in airports:
    # the url, querystring and headers were created in rapidapi website and copied here
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport}/2023-03-28T20:00/2023-03-29T08:00"
    querystring = {"direction":"Arrival","withCancelled":"true","withCodeshared":"false","withLocation":"false"}
    
    headers = {
        "X-RapidAPI-Key": config.flights_API_key,
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }
    # make the request
    response = requests.request("GET", url, headers=headers, params=querystring)
    # get the json data
    json_data = response.json()['arrivals']
    
    for flight in json_data:
        # if flight['movement']['airport'].get("icao") in airports:
            arrivals_dict['icao'].append(flight['movement']['airport'].get("icao", None))
            arrivals_dict['city'].append(flight['movement']['airport'].get('name', None))
            arrivals_dict['actual_arrival'].append(flight['movement'].get('scheduledTimeUtc', None))
            arrivals_dict['scheduled_arrival'].append(flight['movement'].get('scheduledTimeLocal', None))
            arrivals_dict['terminal'].append(flight['movement'].get('terminal', None))
            arrivals_dict['airline'].append(flight['airline'].get('name', None))
            arrivals_dict['airport'].append(airport)  

arrivals_df = pd.DataFrame(arrivals_dict)

In [215]:
# here indentation prevents all airports from being added to the dataframe. The last airport LTFM overwrites the former airports in the list

# for flight in json_data:
#     arrivals_dict['icao'].append(flight['movement']['airport'].get("icao", None))
#     arrivals_dict['city'].append(flight['movement']['airport'].get('name', None))
#     arrivals_dict['actual_arrival'].append(flight['movement'].get('scheduledTimeUtc', None))
#     arrivals_dict['scheduled_arrival'].append(flight['movement'].get('scheduledTimeLocal', None))
#     arrivals_dict['terminal'].append(flight['movement'].get('terminal', None))
#     arrivals_dict['airline'].append(flight['airline'].get('name', None))
#     arrivals_dict['airport'].append(airport)  

# arrivals_df = pd.DataFrame(arrivals_dict)

In [37]:
arrivals_df.sample(30)


,icao,city,actual_arrival,scheduled_arrival,terminal,airline,airport
94,EDDV,Hanover,2023-03-28 22:46Z,2023-03-29 00:46+02:00,None,Unknown/Private owner,EDDF
222,LUKK,Chişinău,2023-03-28 19:45Z,2023-03-28 22:45+03:00,None,Turkish,LTFM
140,OOMS,Muscat,2023-03-29 05:05Z,2023-03-29 07:05+02:00,2,Oman Air,EDDF
110,KCLT,Charlotte,2023-03-29 05:20Z,2023-03-29 07:20+02:00,2,American,EDDF
66,LTFM,Istanbul,2023-03-28 19:00Z,2023-03-28 21:00+02:00,1,Turkish,EDDF
354,RJTT,Tokyo,2023-03-29 02:15Z,2023-03-29 05:15+03:00,None,Turkish,LTFM
295,LTBS,Dalaman,2023-03-28 21:05Z,2023-03-29 00:05+03:00,None,Turkish,LTFM
155,None,Medinah,2023-03-28 17:45Z,2023-03-28 20:45+03:00,None,Pakistan International,LTFM
301,None,Adana,2023-03-28 21:45Z,2023-03-29 00:45+03:00,None,Turkish,LTFM
158,LTAR,Sivas,2023-03-28 17:50Z,2023-03-28 20:50+03:00,None,Turkish,LTFM


In [42]:
arrivals_port_dict = {
    'icao': [],
    'city': [],
    'actual_arrival': [],
    'scheduled_arrival': [],
    'terminal': [],
    'airline': [],
    'airport': []
}

for flight in json_data:
    # if flight['movement']['airport'].get("icao") in airports:
        arrivals_port_dict['icao'].append(flight['movement']['airport'].get("icao", None))
        arrivals_port_dict['city'].append(flight['movement']['airport'].get('name', None))
        arrivals_port_dict['actual_arrival'].append(flight['movement'].get('scheduledTimeUtc', None))
        arrivals_port_dict['scheduled_arrival'].append(flight['movement'].get('scheduledTimeLocal', None))
        arrivals_port_dict['terminal'].append(flight['movement'].get('terminal', None))
        arrivals_port_dict['airline'].append(flight['airline'].get('name', None))
        arrivals_port_dict['airport'].append(airport)  

arrivals_port_df = pd.DataFrame(arrivals_dict)

In [44]:
arrivals_port_df.sample(30)

,icao,city,actual_arrival,scheduled_arrival,terminal,airline,airport
310,LTCA,Elazığ,2023-03-28 22:30Z,2023-03-29 01:30+03:00,None,Turkish,LTFM
108,VVTS,Ho Chi Minh City,2023-03-29 04:30Z,2023-03-29 06:30+02:00,2,Vietnam,EDDF
375,OOMS,Muscat,2023-03-29 02:35Z,2023-03-29 05:35+03:00,None,Turkish,LTFM
206,EPWA,Warsaw,2023-03-28 19:40Z,2023-03-28 22:40+03:00,None,Turkish,LTFM
244,EGCC,Manchester,2023-03-28 20:15Z,2023-03-28 23:15+03:00,None,Turkish,LTFM
385,None,Adana,2023-03-28 23:55Z,2023-03-29 02:55+03:00,None,Turkish,LTFM
196,LWSK,Skopje,2023-03-28 19:45Z,2023-03-28 22:45+03:00,None,Turkish,LTFM
257,EDDB,Berlin,2023-03-28 20:20Z,2023-03-28 23:20+03:00,None,Turkish,LTFM
36,EDDB,Berlin,2023-03-28 18:55Z,2023-03-28 20:55+02:00,1,Lufthansa,EDDF
29,LEBB,Bilbao,2023-03-28 18:55Z,2023-03-28 20:55+02:00,1,Lufthansa,EDDF


time aggregation and function

In [75]:
#import the libraries for datetime and timezone
from datetime import datetime, date, timedelta
from pytz import timezone

In [76]:
#setting the timezone to Berlin and getting the current date
today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
today

datetime.date(2023, 4, 2)

In [77]:
#creating a variable for tomorrow
tomorrow = (today + timedelta(days=1))
tomorrow           

datetime.date(2023, 4, 3)

In [124]:
#THIS VERSION HAS aggregated two 12 hour periods into one day

# icao codes for airports frankfurt, istanbul
airports = ['EDDF', 'LTFM']

arrivals_dict = {
    'departure_icao': [],
    'city': [],
    'actual_arrival': [],
    'scheduled_arrival': [],
    'terminal': [],
    'airline': [],
    'arrival_icao': []
    }
list_for_df = []

for airport in airports:
    times = [["00:00","11:59"],["12:00","23:59"]]
    # loop through the airports list
    for time in times:
        # the url, querystring and headers were created in rapidapi website and copied here
        #uncomment the below lines when you want to refressh the data and json response
        
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
        querystring = {"direction":"Arrival","withCancelled":"true","withCodeshared":"false","withLocation":"false"}
        
        headers = {
            "X-RapidAPI-Key": config.flights_API_key,
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }
        # make the request
        response = requests.request("GET", url, headers=headers, params=querystring)
        # get the json data
        time_json_data = response.json()['arrivals']
        
        for flight in time_json_data:
            # if flight['movement']['airport'].get("icao") in airports:
                arrivals_dict['departure_icao'].append(flight['movement']['airport'].get("icao", None))
                arrivals_dict['city'].append(flight['movement']['airport'].get('name', None))
                arrivals_dict['actual_arrival'].append(flight['movement'].get('scheduledTimeUtc', None))
                arrivals_dict['scheduled_arrival'].append(flight['movement'].get('scheduledTimeLocal', None))
                arrivals_dict['terminal'].append(flight['movement'].get('terminal', None))
                arrivals_dict['airline'].append(flight['airline'].get('name', None))
                arrivals_dict['arrival_icao'].append(airport)  

arrivals_df = pd.DataFrame(arrivals_dict)

In [104]:
arrivals_df.sample(30)

,icao,city,actual_arrival,scheduled_arrival,terminal,airline,airport
43,URSS,Sochi,2023-04-03 10:40Z,2023-04-03 13:40+03:00,None,Azimuth,EDDF
1555,LSZH,Zurich,2023-04-03 19:30Z,2023-04-03 22:30+03:00,None,Turkish,LTFM
646,LTCC,Diyarbakır,2023-04-03 15:25Z,2023-04-03 18:25+03:00,None,Turkish,EDDF
1225,ORER,Arbil,2023-04-03 09:40Z,2023-04-03 12:40+03:00,None,I A W,LTFM
1487,UGTB,Tbilisi,2023-04-03 16:05Z,2023-04-03 19:05+03:00,None,Turkish,LTFM
649,LTCT,Iğdır,2023-04-03 15:35Z,2023-04-03 18:35+03:00,None,Turkish,EDDF
1172,EDDL,Duesseldorf,2023-04-03 20:05Z,2023-04-03 23:05+03:00,None,Turkish,LTFM
601,LTAZ,Nevşehir,2023-04-03 14:00Z,2023-04-03 17:00+03:00,None,Turkish,EDDF
151,ENGM,Oslo,2023-04-03 13:45Z,2023-04-03 16:45+03:00,None,Turkish,EDDF
129,LSGG,Geneva,2023-04-03 12:10Z,2023-04-03 15:10+03:00,None,Turkish,EDDF


In [125]:
arrivals_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264 entries, 0 to 1263
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   icao               1154 non-null   object
 1   city               1264 non-null   object
 2   actual_arrival     1264 non-null   object
 3   scheduled_arrival  1264 non-null   object
 4   terminal           563 non-null    object
 5   airline            1264 non-null   object
 6   airport            1264 non-null   object
dtypes: object(7)
memory usage: 69.2+ KB


In [129]:
arrivals_df['actual_arrival'] = pd.to_datetime(arrivals_df['actual_arrival'], utc=True)
arrivals_df['scheduled_arrival'] = pd.to_datetime(arrivals_df['scheduled_arrival'], utc=True)
arrivals_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264 entries, 0 to 1263
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   icao               1154 non-null   object             
 1   city               1264 non-null   object             
 2   actual_arrival     1264 non-null   datetime64[ns, UTC]
 3   scheduled_arrival  1264 non-null   datetime64[ns, UTC]
 4   terminal           563 non-null    object             
 5   airline            1264 non-null   object             
 6   airport            1264 non-null   object             
dtypes: datetime64[ns, UTC](2), object(5)
memory usage: 69.2+ KB


In [ ]:
# only keep the flights that are scheduled to arrive today
# arrivals_df = arrivals_df[arrivals_df['scheduled_arrival'].dt.date == today]

# convert the actual arrival time to Berlin timezone
# arrivals_df['actual_arrival'] = arrivals_df['actual_arrival'].dt.tz_convert('Europe/Berlin')


push the DataFrame to MySQL

In [118]:
# we need sqlalchemy to connect to the database
# %pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [119]:
import sqlalchemy

In [120]:
#Installed the below package is necessary to run SQL alchemy
# %pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [122]:
# improting a placeholder for the MySQL password
from config import password
#in schema use the name of database not of DF
schema="weather_flight_pipelines"   # name of the database you want to use here
host="127.0.0.1"        # to connect to your local server
user="root"
password=config.password # your password!!!!
port=3306
connection_details = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [158]:
# push the dataframe to the MySQL database

#here use the name of DF you want to push
airports_df.to_sql('airports',         # 'iss_logs'-> table name;
              if_exists='append', # if_exists -> will create new table if doesn't exist, otherwise, 'append' - will append data to existing table;
              con=connection_details,            # con-> connection string;
              index=False) 

2

In [139]:
# push the dataframe to the MySQL database

#here use the name of DF you want to push
arrivals_df.to_sql('arrivals',         # 'iss_logs'-> table name;
              if_exists='append', # if_exists -> will create new table if doesn't exist, otherwise, 'append' - will append data to existing table;
              con=connection_details,            # con-> connection string;
              index=False) 

1264

In [138]:
#the 22nd row was longer than 'city' varchar in the database
city_value = arrivals_df.iloc[21]['city']
len(city_value)

42

CREATING FUNCTION

In [96]:
#Function to get the data from the API

def tomorrow_arrivals(airports):

    today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
    tomorrow = (today + timedelta(days=1))
    arrivals_dict = {
        'departure_icao': [],
        'city': [],
        'actual_arrival': [],
        'scheduled_arrival': [],
        'terminal': [],
        'airline': [],
        'arrival_icao': []
        }

    for airport in airports:
        times = [["00:00","11:59"],["12:00","23:59"]]
        # loop through the airports list
        for time in times:
            # the url, querystring and headers were created in rapidapi website and copied here
            
            
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
            querystring = {"direction":"Arrival","withCancelled":"true","withCodeshared":"false","withLocation":"false"}
            
            headers = {
                "X-RapidAPI-Key": config.flights_API_key,
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }
            # make the request
            response = requests.request("GET", url, headers=headers, params=querystring)
            # get the json data
            time_json_data = response.json()['arrivals']
            
            for flight in time_json_data:
                # if flight['movement']['airport'].get("icao") in airports:
                    arrivals_dict['departure_icao'].append(flight['movement']['airport'].get("icao", None))
                    arrivals_dict['city'].append(flight['movement']['airport'].get('name', None))
                    arrivals_dict['actual_arrival'].append(flight['movement'].get('scheduledTimeUtc', None))
                    arrivals_dict['scheduled_arrival'].append(flight['movement'].get('scheduledTimeLocal', None))
                    arrivals_dict['terminal'].append(flight['movement'].get('terminal', None))
                    arrivals_dict['airline'].append(flight['airline'].get('name', None))
                    arrivals_dict['arrival_icao'].append(airport)  

    arrivals_df = pd.DataFrame(arrivals_dict)
    return arrivals_df

In [133]:
airports = ['EDDF', 'LTFM'] # the function makes 4 api calls although there are only 2 airports in the list
tomorrow_arrivals(airports).sample(40)

,icao,city,actual_arrival,scheduled_arrival,terminal,airline,airport
974,None,Shirnak,2023-04-04 15:25Z,2023-04-04 18:25+03:00,None,Turkish,LTFM
1020,LTBJ,İzmir,2023-04-04 17:10Z,2023-04-04 20:10+03:00,None,Turkish,LTFM
869,KSEA,Seattle,2023-04-04 12:30Z,2023-04-04 15:30+03:00,None,Turkish,LTFM
392,LEMD,Madrid,2023-04-04 15:50Z,2023-04-04 17:50+02:00,2,Air Europa,EDDF
1128,LEMD,Madrid,2023-04-04 20:30Z,2023-04-04 23:30+03:00,None,Turkish,LTFM
481,EDDH,Hamburg,2023-04-04 18:10Z,2023-04-04 20:10+02:00,1,Lufthansa,EDDF
85,EBBR,Brussels,2023-04-04 06:10Z,2023-04-04 08:10+02:00,1,Brussels,EDDF
792,None,Ulaan Baatar,2023-04-04 10:05Z,2023-04-04 13:05+03:00,None,Turkish,LTFM
480,LIRF,Rome,2023-04-04 18:25Z,2023-04-04 20:25+02:00,1,Lufthansa,EDDF
687,LTAU,Kayseri,2023-04-04 07:15Z,2023-04-04 10:15+03:00,None,Turkish,LTFM


working experiments
